In [ ]:
import pandas as pd
from graph_tool.all import *


In [ ]:
# set filename
filename = "assoSongExp.h5"

In [ ]:
# # build graph 

# read data from a hdf5 file
df = pd.read_hdf(filename, key='asso', mode='r')
# first [0] and second [1] columns are nodes, the third [2] column is weight

# get first two columns to build network
testsamples = df.iloc[:,0:2].values

# declare the graph
pureG = Graph(directed=True)

# using the pair of nodes to build a network (without weight yet)
# Since we used hash, the returning object is a map storing the name of each vertex
namemap = pureG.add_edge_list(testsamples, hashed=True, string_vals=False)

# incorporate the map into the graph as a property
pureG.vertex_properties['name'] = namemap

# save the unweighted graph with a map of name
pureG.save("networkwithname.xml.gz")


# declare a property map of edges to store weight
wtmp = pureG.new_edge_property("double")

# assign the third [2] column to the weight map
wtmp.a = df.iloc[:,2]
# incorporate the weight map into the graph as a property
pureG.edge_properties['weight'] = wtmp

# save weight along with the network
pureG.save("networkwithweight.xml.gz")

namelist= []
for i in namemap:
    namelist.append(i)

In [ ]:
# #Closeness
cloG = Graph(directed=True)

# load the network
cloG.load("networkwithweight.xml.gz")

# declare property map of vertex to store closeness of vertex
clo_vert = cloG.new_vertex_property("double")

graph_tool.centrality.closeness(cloG, weight=cloG.edge_properties['weight'], source=None, vprop=clo_vert, norm=True, harmonic=False)

cloG.vertex_properties['VertexCloseness'] = clo_vert

# save the final graph
cloG.save("networkwithclo.xml.gz")

clodf = pd.DataFrame({'code':namelist,'closeness':clo_vert.a})
clodf.to_hdf('clodf.h5', key='asso', mode='w')

In [ ]:
# #Betweeness
from graph_tool.all import *

betG = Graph(directed=True)

# load the network
betG.load("networkwithweight.xml.gz")

# declare property map of vertex to store betweenness of vertex
bet_vert = betG.new_vertex_property("double")
# declare property map of edge to store betweenness of edge
bet_edge = betG.new_edge_property("double")

# Calculate the betweenness; Using wtmp as weight, 
# return the betweenness of vertex to bet_vert, 
# return the betweenness of edge to bet_edge,
graph_tool.centrality.betweenness(betG, pivots=None, vprop=bet_vert, eprop=bet_edge, weight=betG.edge_properties['weight'], norm=False)

# incorprate the betweeness map (both edge and vertex) into the graph
betG.edge_properties['EdgeBetweenness'] = bet_edge
betG.vertex_properties['VertesBetweenness'] = bet_vert

# save the final graph
betG.save("networkwithbet.xml.gz")

betdf = pd.DataFrame({'code':namelist,'betweenness':bet_vert.a})
betdf.to_hdf('betdf.h5', key='asso', mode='w')

In [ ]:
# #Eigenvector Still debugging, but can use networkX version instead
eigG = Graph(directed=True)

# load the network
eigG.load("networkwithweight.xml.gz")

# declare property map of vertex to store eigenvector of vertex
ee, eigp = eigenvector(eigG, weight = eigG.edge_properties['weight'])

eigG.vertex_properties['VertesEigenvector'] = eigp

# save the final graph
eigG.save("networkwitheig.xml.gz")

eigdf = pd.DataFrame({'code':namelist,'VertesEigenvector':eigp.a})

eigdf.to_hdf('eigdf.h5', key='asso', mode='w')